<a href="https://colab.research.google.com/github/gokuno1/Excel/blob/new/PricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from sklearn.svm import SVR
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [29]:
#Load the data
from google.colab import files
files.upload()

Saving IOC.csv to IOC (1).csv


{'IOC.csv': b'Date,Close Pirce,EMA,RSI,OBV,ADX,MACD,STCH MTM,M FLOW,,\r\n4/16/2021,90.15,94.63,41,395000000,31,-1.5,-28,43,,\r\n4/19/2021,88.1,94.37,37,379000000,32,-2.3,-33,46,,\r\n4/20/2021,87.8,94.12,37.02,368000000,31.81,-2.26,-46.35,53.49,,\r\n4/22/2021,87.85,93.87,37,378000000,33,-2.4,-52,57,,\r\n4/26/2021,88.2,93.43,38.11,393000000,31.14,-2.22,-50.33,53.25,,\r\n4/27/2021,88.9,93.25,40.82,402000000,30.61,-2.19,-49.84,61.16,,\r\n4/28/2021,89.3,93.09,42.39,408000000,29.82,-2.06,-46.85,66.83,,\r\n4/29/2021,88.75,92.92,40.9,399000000,28.48,-1.81,-42.37,60.14,,\r\n5/3/2021,91.5,92.8,50.83,443000000,25.69,-1.33,-26.04,75.5,,\r\n5/4/2021,91.5,92.8,50.84,457000000,25.16,-1.08,-16.98,86.11,,\r\n5/5/2021,92.35,92.84,53.83,451000000,24.67,-0.96,-7.56,86.72,,\r\n5/6/2021,92.75,92.84,55.24,465000000,29.11,-0.72,2.24,86.68,,\r\n5/7/2021,94.7,92.79,61.32,484000000,24.89,-0.36,12.62,87.31,,\r\n5/10/2021,99.2,93.04,71.03,521000000,26.37,0.38,24.06,95.71,,\r\n5/11/2021,103.55,93.45,77.04,598000000

In [30]:
df = pd.read_csv('IOC.csv')
df

,Date,Close Pirce,EMA,RSI,OBV,ADX,MACD,STCH MTM,M FLOW,Unnamed: 9,Unnamed: 10
0,4/16/2021,90.15,94.63,41.00,395000000,31.00,-1.50,-28.00,43.00,NaN,NaN
1,4/19/2021,88.10,94.37,37.00,379000000,32.00,-2.30,-33.00,46.00,NaN,NaN
2,4/20/2021,87.80,94.12,37.02,368000000,31.81,-2.26,-46.35,53.49,NaN,NaN
3,4/22/2021,87.85,93.87,37.00,378000000,33.00,-2.40,-52.00,57.00,NaN,NaN
4,4/26/2021,88.20,93.43,38.11,393000000,31.14,-2.22,-50.33,53.25,NaN,NaN
5,4/27/2021,88.90,93.25,40.82,402000000,30.61,-2.19,-49.84,61.16,NaN,NaN
6,4/28/2021,89.30,93.09,42.39,408000000,29.82,-2.06,-46.85,66.83,NaN,NaN
7,4/29/2021,88.75,92.92,40.90,399000000,28.48,-1.81,-42.37,60.14,NaN,NaN
8,5/3/2021,91.50,92.80,50.83,443000000,25.69,-1.33,-26.04,75.50,NaN,NaN
9,5/4/2021,91.50,92.80,50.84,457000000,25.16,-1.08,-16.98,86.11,NaN,NaN


In [ ]:
#Show and store last line of data
actual_price = df.tail(1)
#print data
actual_price


,Date,Close Pirce,EMA,RSI,OBV,ADX,MACD,STCH MTM,M FLOW,Unnamed: 9,Unnamed: 10
15,5/12/2021,102.7,93.82,73.46,559000000,31.35,1.82,42.39,100.0,NaN,NaN


In [31]:
#Remove last line and show remaining data
df = df.head(len(df)-1)
df

,Date,Close Pirce,EMA,RSI,OBV,ADX,MACD,STCH MTM,M FLOW,Unnamed: 9,Unnamed: 10
0,4/16/2021,90.15,94.63,41.00,395000000,31.00,-1.50,-28.00,43.00,NaN,NaN
1,4/19/2021,88.10,94.37,37.00,379000000,32.00,-2.30,-33.00,46.00,NaN,NaN
2,4/20/2021,87.80,94.12,37.02,368000000,31.81,-2.26,-46.35,53.49,NaN,NaN
3,4/22/2021,87.85,93.87,37.00,378000000,33.00,-2.40,-52.00,57.00,NaN,NaN
4,4/26/2021,88.20,93.43,38.11,393000000,31.14,-2.22,-50.33,53.25,NaN,NaN
5,4/27/2021,88.90,93.25,40.82,402000000,30.61,-2.19,-49.84,61.16,NaN,NaN
6,4/28/2021,89.30,93.09,42.39,408000000,29.82,-2.06,-46.85,66.83,NaN,NaN
7,4/29/2021,88.75,92.92,40.90,399000000,28.48,-1.81,-42.37,60.14,NaN,NaN
8,5/3/2021,91.50,92.80,50.83,443000000,25.69,-1.33,-26.04,75.50,NaN,NaN
9,5/4/2021,91.50,92.80,50.84,457000000,25.16,-1.08,-16.98,86.11,NaN,NaN


In [32]:
#create empty list
days_data = list()
close_price = list()
RSI = list()
adx = list()
macd = list()
ema = list()

In [33]:
df_days = df.loc[:, 'Date']
df_close_price = df.loc[:, 'Close Pirce']
df_rsi = df.loc[:, 'RSI']
df_adx = df.loc[:, 'ADX']
df_macd = df.loc[:, 'MACD']
df_ema = df.loc[:, 'EMA']


In [34]:
#create independant data set of dates and dependeant other
for day in df_days:
  days_data.append([int(day.split('/')[1])])

for closes in df_close_price:
  close_price.append(float(closes))

for rsis in df_rsi:
  RSI.append(float(rsis))

for adxs in df_adx:
  adx.append(float(adxs))

for macds in df_macd:
  macd.append(float(macds))

for emas in df_ema:
  ema.append(float(emas))


In [35]:
days_data

[[16],
 [19],
 [20],
 [22],
 [26],
 [27],
 [28],
 [29],
 [3],
 [4],
 [5],
 [6],
 [7],
 [10],
 [11]]

In [ ]:
# Show the data
adx

[31.0,
 32.0,
 31.81,
 33.0,
 31.14,
 30.61,
 29.82,
 28.48,
 25.69,
 25.16,
 24.67,
 29.11,
 24.89,
 26.37,
 28.8]

In [36]:
#Create Model
rbf_model = SVR(kernel='rbf', C=1000.0, gamma=0.85)
#rbf_model.fit(days_data, close_price, adx) 
#rbf_model.fit(days_data, close_price, RSI) 
rbf_model.fit(days_data, close_price, macd) 

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.85,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#plot the data
plt.figure(figsize=(16,8))
plt.scatter(days, close_price, RSI, color = 'black', label='Data')

In [37]:
day = [[17]]
print('Model prediction: ', rbf_model.predict(day))

Model prediction:  [101.375]
